In [41]:
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

## get graph of interest

In [82]:
place_name = "Berlin, Germany"
#place_name = "Grunewald, Berlin, Germany"

# Fetch OSM street network from the location
place_graph = ox.graph_from_place(place_name, network_type='bike', simplify=True)

In [ ]:
# Plot the streets
fig, ax = ox.plot_graph(place_graph,edge_linewidth=1, node_size=0, figsize=(16,16),
                        bgcolor='#ffffff', edge_color='black', node_color='black'
                        )

## remove tracks from graph

In [ ]:
# remove ways without names
graph_nodes, graph_edges = ox.graph_to_gdfs(place_graph)

graph_edges = graph_edges.query('~name.isnull()')

place_graph = ox.graph_from_gdfs(graph_nodes, graph_edges)
place_graph = ox.utils_graph.remove_isolated_nodes(place_graph)
place_graph = ox.utils_graph.get_largest_component(place_graph)

In [ ]:
# Plot the streets
fig, ax = ox.plot_graph(place_graph,edge_linewidth=1, node_size=3, figsize=(16,16),
                        bgcolor='#ffffff', edge_color='black', node_color='red'
                        )

## stats

In [ ]:
stats = ox.stats.basic_stats(graph)
print("number of nodes: "+ stats['n'])
print("number of steets: "+ stats['m'])
print("total steetlength: "+ stats['edge_length_total'] +"m")
print("durchschnittliche Straßenlänge: "+ stats['edge_length_avg'] +"m")

In [57]:
ox.plot_figure_ground(place, edge_linewidth=0, node_size=1, figsize=(16,16),
                        bgcolor='#ffffff', edge_color='black', node_color='black')

KeyboardInterrupt: 

In [21]:
nodes, streets = ox.graph_to_gdfs(graph)
print("nodes")
print(nodes)
print("steets")
print(streets)

nodes
                    y          x  street_count highway  ref  \
osmid                                                         
172539      52.565205  13.335501             3     NaN  NaN   
172545      52.567033  13.346605             4     NaN  NaN   
172546      52.567738  13.352255             3     NaN  NaN   
172547      52.564077  13.339564             3     NaN  NaN   
172549      52.563398  13.342901             3     NaN  NaN   
...               ...        ...           ...     ...  ...   
8548027510  52.593559  13.460480             1     NaN  NaN   
8548061529  52.606397  13.448658             1     NaN  NaN   
8548065815  52.591479  13.453175             1     NaN  NaN   
8548170453  52.627561  13.489100             1     NaN  NaN   
8548234159  52.616005  13.492682             1     NaN  NaN   

                             geometry  
osmid                                  
172539      POINT (13.33550 52.56521)  
172545      POINT (13.34661 52.56703)  
172546      PO